# 3: Data cleanup

dt


- To save new data, do the following:
    - define `merged_data_path` and `raw_data_path`
    - define `get_raw_data` and `get_merged_data`
    - run `tweets_df = get_merged_data(kind = 'tweets', numbers = range( NUMBER ))` with NUMBER the maximum csv number
    - run `tweets_df.to_csv(merged_data_path + 'merged_tweets.csv')`
    - define `fix_user_index`
    - run the loop for `fix_user_index(user_number)`
    - run `users_df = get_merged_data(kind = 'users_idfixed', numbers = range( NUMBER ))`
    - run `users_df.to_csv(merged_data_path + 'merged_users.csv')`
This creates a couple large files `/merged_data/merged_tweets.csv` and `/merged_data/merged_users.csv`. These should be ran on a local machine and `.gitignore`-d.

The data are saved in `/raw_data/` with tweet data in `tweets_NN.csv` and user data for those tweets in `old_users/users_NN.csv`. The user data has an error in saving the user id, so this notebook also corrects that and saves them as `raw_data/users_idfixed_NN.csv`.

The can be merged for user info on `tweets.author_id = users.id` in a later notebook.

- [First results](#First-results)
    - [first tweet results](#first-tweet-results)
    - [first user results](#first-user-results)
    - [merging](#merging)
- [Merging and saving tweet data](#Merging-and-saving-tweet-data)
- [Merging and saving user data](#Merging-and-saving-user-data)
    - [Fixing user data ids](#Fixing-user-data-ids)
    - [Merging and saving user data](#Merging-and-saving-user-data)
- [Saving Wordle text](#Saving-Wordle-text)

In [1]:
import pandas as pd
import numpy as np
import os

## First results

Starting by pulling in the first results and trying a merge.

In [2]:
raw_data_path = '../raw_data/'
tweets_0_name = 'tweets_0.csv'
users_0_name = 'raw_users/users_0.csv'

t0 = pd.read_csv(raw_data_path + tweets_0_name, index_col = 'id')

### first tweet results

In [51]:

t0.head()

,conversation_id,reply_settings,lang,entities,possibly_sensitive,source,public_metrics,text,context_annotations,author_id,created_at,referenced_tweets,in_reply_to_user_id,attachments,geo
id,,,,,,,,,,,,,,,
1521733018293084160,1521733018293084160,everyone,en,"{'urls': [{'start': 54, 'end': 77, 'url': 'htt...",False,Twitter Web App,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Wordle (ES) #118 5/6\n\n⬜⬜🟨🟩⬜\n⬜⬜🟨🟩⬜\n⬜⬜⬜🟩🟩\n...,"[{'domain': {'id': '30', 'name': 'Entities [En...",1006110502043471872,2022-05-04T06:06:38.000Z,NaN,NaN,NaN,NaN
1521733015738662913,1521733015738662913,everyone,es,"{'urls': [{'start': 58, 'end': 81, 'url': 'htt...",False,Twitter for Android,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Wordle Científico #52 5/6\n\n⬜⬜⬜⬜🟨\n🟨⬜⬜⬜🟩\n⬜🟩⬜...,"[{'domain': {'id': '30', 'name': 'Entities [En...",360002014,2022-05-04T06:06:38.000Z,NaN,NaN,NaN,NaN
1521733011414257664,1521733011414257664,everyone,en,NaN,False,Twitter for Android,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Wordle 319 4/6\n\n🟨🟨⬜⬜⬜\n🟨⬜⬜🟩⬜\n🟩🟨⬜🟩🟨\n🟩🟩🟩🟩🟩,"[{'domain': {'id': '30', 'name': 'Entities [En...",828818052892422148,2022-05-04T06:06:37.000Z,NaN,NaN,NaN,NaN
1521733010504445954,1521733010504445954,everyone,en,NaN,False,Twitter for Android,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",Back in the saddle.\n\nWordle 319 3/6\n\n⬜🟩⬜🟨⬜...,"[{'domain': {'id': '30', 'name': 'Entities [En...",862077942,2022-05-04T06:06:36.000Z,NaN,NaN,NaN,NaN
1521733001981444096,1521731925559418880,everyone,en,"{'mentions': [{'start': 0, 'end': 12, 'usernam...",False,Twitter for iPhone,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",@RwellsWells Same. Are we both spotters? Haha ...,"[{'domain': {'id': '30', 'name': 'Entities [En...",53116513,2022-05-04T06:06:34.000Z,"[{'type': 'replied_to', 'id': '152173192555941...",331622540.0,NaN,NaN


In [8]:
t0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1997 entries, 1521733018293084160 to 1521721821825015809
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   conversation_id      1997 non-null   int64  
 1   reply_settings       1997 non-null   object 
 2   lang                 1997 non-null   object 
 3   entities             881 non-null    object 
 4   possibly_sensitive   1997 non-null   bool   
 5   source               1997 non-null   object 
 6   public_metrics       1997 non-null   object 
 7   text                 1997 non-null   object 
 8   context_annotations  1950 non-null   object 
 9   author_id            1997 non-null   int64  
 10  created_at           1997 non-null   object 
 11  referenced_tweets    185 non-null    object 
 12  in_reply_to_user_id  178 non-null    float64
 13  attachments          19 non-null     object 
 14  geo                  45 non-null     object 
dtypes: bo

### first user results

In [51]:
u0 = pd.read_csv(raw_data_path + users_0_name, index_col='id')

In [52]:
u0.head()

,Unnamed: 0,name,public_metrics,url,description,profile_image_url,location,protected,username,verified,entities,pinned_tweet_id
id,,,,,,,,,,,,
1.006111e+18,0,Jose Antonio,"{'followers_count': 147, 'following_count': 43...",NaN,"Antiguo usuario bloqueado por error, pero he v...",https://pbs.twimg.com/profile_images/144681186...,"Alcalá de Guadaíra, Spain",False,Joseamj_97,False,NaN,NaN
3.600020e+08,1,Ana V Garcia,"{'followers_count': 41, 'following_count': 136...",https://t.co/uMfikCmd0I,NaN,https://pbs.twimg.com/profile_images/122215871...,Alcalá de Henares,False,AnaVGarciaG,False,"{'url': {'urls': [{'start': 0, 'end': 23, 'url...",NaN
8.288181e+17,2,Braden Van't Hof,"{'followers_count': 8, 'following_count': 282,...",NaN,LoL Head Coach/Manager/Mid Laner/4x Positional...,https://pbs.twimg.com/profile_images/149527273...,"Grand Rapids, MI",False,CLVRockmelon,False,NaN,NaN
8.620779e+08,3,John Milton ✊,"{'followers_count': 1465, 'following_count': 2...",https://t.co/iOfRLxEru5,"It's not criticism, it's just a different way ...",https://pbs.twimg.com/profile_images/150123920...,Spain,False,Geordioca,False,"{'url': {'urls': [{'start': 0, 'end': 23, 'url...",NaN
5.311651e+07,4,christopher jones,"{'followers_count': 189, 'following_count': 26...",NaN,I don't like cricket. oh no. i love it!,https://pbs.twimg.com/profile_images/110881711...,sheffield,False,chrisjones792,False,NaN,NaN


In [53]:
u0.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 1982 entries, 1.0061105020434719e+18 to nan
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         1982 non-null   int64  
 1   name               1982 non-null   object 
 2   public_metrics     1982 non-null   object 
 3   url                579 non-null    object 
 4   description        1696 non-null   object 
 5   profile_image_url  1982 non-null   object 
 6   location           1449 non-null   object 
 7   protected          1982 non-null   bool   
 8   username           1982 non-null   object 
 9   verified           1982 non-null   bool   
 10  entities           811 non-null    object 
 11  pinned_tweet_id    912 non-null    float64
dtypes: bool(2), float64(1), int64(1), object(8)
memory usage: 174.2+ KB


There's a ~~big~~ problem with how I saved the users data --- there was a missing line in `repeated_twitter_search` I've since added in:
```python
new_users_df.set_index('id', inplace = True)
```
The first 100 records of every `users_xx.csv` data is an integer, and then becomes the id of the user. After this, the `id` column becomes NaNs. This must be rectified!

### merging

I can expand the JSON columns (`entities_t`, `entities_u`, `public_metrics_t`, `context_annotations`, `public_metrics_u`) later.

In [52]:
t0.columns

Index(['conversation_id', 'reply_settings', 'lang', 'entities',
       'possibly_sensitive', 'source', 'public_metrics', 'text',
       'context_annotations', 'author_id', 'created_at', 'referenced_tweets',
       'in_reply_to_user_id', 'attachments', 'geo'],
      dtype='object')

In [53]:
u0.columns

Index(['Unnamed: 0', 'name', 'public_metrics', 'url', 'description',
       'profile_image_url', 'location', 'protected', 'username', 'verified',
       'entities', 'pinned_tweet_id'],
      dtype='object')

In [54]:
t0u0.columns

Index(['index', 'conversation_id', 'reply_settings', 'lang', 'entities_t',
       'possibly_sensitive', 'source', 'public_metrics_t', 'text',
       'context_annotations', 'author_id', 'created_at', 'referenced_tweets',
       'in_reply_to_user_id', 'attachments', 'geo', 'Unnamed: 0', 'name',
       'public_metrics_u', 'url', 'description', 'profile_image_url',
       'location', 'protected', 'username', 'verified', 'entities_u',
       'pinned_tweet_id'],
      dtype='object')

Rather than combining each tweetdf with each userdf, then tacking them on to an aggregated df, let's do this the table / SQL way. Keep the users and tweets separate. They can be joined when necessary. This will save some space. This seems to be the way that Twitter suggests anyway, with the distinction between `tweet.fields` and `user.fields` in the search parameters.

## Merging and saving tweet data

Go through each tweets or users csv in `raw_data`, concatenate them, and save the result.

In [3]:
merged_data_path = '../merged_data/'
raw_data_path = '../raw_data/'

#loads the data in a csv and returns a dataframe with it.
# can get tweet data or user data.

def get_raw_data(kind = 'tweets', number = 0):
    new_df = pd.read_csv(f'{raw_data_path}{kind}_{number}.csv')
    new_df.set_index('id', inplace = True)
    return new_df

# repeatedly call the previous, tacking each onto the end of a new dataframe,
# and return the dataframe


def get_merged_data(kind = 'tweets', numbers = [0,1]):
    #make a new blank df
    new_df = pd.DataFrame()
    #grab each file and tack data onto the end of the dataframe
    for number in numbers:
        this_df = get_raw_data(kind = kind, number = number)
        if kind == 'tweets':
            new_df = pd.concat([new_df, this_df])
        else: #users or users_idfixed
            #explicitly making it 'outer' may be unneccesary. Just making sure
            # there are no duplicates.
            new_df = pd.concat([new_df, this_df], join = 'outer')
    #convert created_at string to datetime before returning
    if 'created_at' in new_df.columns:
        new_df['created_at'] = pd.to_datetime(new_df['created_at'])
    return new_df


# using the above function for both. no need for this.
# def get_merged_tweets(numbers = [0,1]):
#     #make a new blank df
#     new_df = pd.DataFrame()
#     #grab each file and tack data onto the end of the dataframe
#     for number in numbers:
#         this_df = get_raw_data(kind = kind, number = number)
#         if kind == 'tweets':
#             new_df = pd.concat([new_df, this_df])
#         else:
#             this_df.drop(columns = ['Unnamed: 0'], inplace = True)
#             new_df = pd.concat([new_df,this_df], join = 'outer')
#     #convert created_at string to datetime before returning
#     if 'created_at' in new_df.columns:
#         new_df['created_at'] = pd.to_datetime(new_df['created_at'])
#     return new_df

In [4]:
%%time
tweets_df = get_merged_data(kind = 'tweets', numbers = range(84+1))

Wall time: 6.02 s


In [82]:
get_raw_data(kind = 'users', number = 17)

,Unnamed: 0,verified,protected,profile_image_url,url,username,public_metrics,description,name,pinned_tweet_id,location,entities
id,,,,,,,,,,,,
2.818584e+09,0,False,False,https://pbs.twimg.com/profile_images/140126251...,NaN,ronylito76,"{'followers_count': 336, 'following_count': 97...",solo estare con los ke conmigo kieran estar\na...,ronylito,1.349024e+18,NaN,NaN
4.286845e+07,1,False,False,https://pbs.twimg.com/profile_images/131140926...,https://t.co/lByLfqg3Y6,ValNgugi,"{'followers_count': 3707, 'following_count': 1...",Marketer. Fixer.,Valerie.,1.121352e+18,"Nairobi, Kenya","{'url': {'urls': [{'start': 0, 'end': 23, 'url..."
9.441338e+08,2,False,False,https://pbs.twimg.com/profile_images/125037546...,NaN,Bio_MER,"{'followers_count': 844, 'following_count': 18...",1 artık yalancıyı s****ler\n2 oy kullanmaya ze...,Endoplazmik1kulum🧠🫀🌳🌲🏞️🌈,1.422639e+18,"New Hampshire, USA",NaN
1.773261e+06,3,False,False,https://pbs.twimg.com/profile_images/477652837...,NaN,CraigKocur,"{'followers_count': 237, 'following_count': 55...",One line Bio?! How is that possible? A whole p...,Craig,NaN,"North Dr Saginaw, MI 48601",NaN
3.391349e+09,4,False,False,https://pbs.twimg.com/profile_images/144695477...,NaN,zubeldianoOK,"{'followers_count': 979, 'following_count': 16...",Liberal 🐍 Pincharrata 🇦🇹 Datos y resultados. E...,Zubeldiano,8.891806e+17,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,186648261,False,False,https://pbs.twimg.com/profile_images/142783651...,https://t.co/PVaMfejUlJ,Phillip_In_HK,"{'followers_count': 1526, 'following_count': 2...","Born in 🏴󠁧󠁢󠁳󠁣󠁴󠁿. 🇭🇰 has been my home, since 19...",Currently travelling out of Hong Kong.,NaN,"Hong Kong, where else?","{'url': {'urls': [{'start': 0, 'end': 23, 'url..."
NaN,82343519,False,False,https://pbs.twimg.com/profile_images/476421839...,NaN,currypuffs,"{'followers_count': 225, 'following_count': 13...",“@currypuffs ur header is still from the kim k...,YX,9.249542e+17,NaN,"{'description': {'mentions': [{'start': 1, 'en..."
NaN,3352174174,False,False,https://pbs.twimg.com/profile_images/151030791...,https://t.co/WhtkgTxG3p,areejpanju,"{'followers_count': 187, 'following_count': 20...",🌸,areej,1.387906e+18,"London, England","{'url': {'urls': [{'start': 0, 'end': 23, 'url..."


In [178]:
tweets_df['text'].head()

id
1521733018293084160    Wordle (ES)  #118 5/6\n\n⬜⬜🟨🟩⬜\n⬜⬜🟨🟩⬜\n⬜⬜⬜🟩🟩\n...
1521733015738662913    Wordle Científico #52 5/6\n\n⬜⬜⬜⬜🟨\n🟨⬜⬜⬜🟩\n⬜🟩⬜...
1521733011414257664         Wordle 319 4/6\n\n🟨🟨⬜⬜⬜\n🟨⬜⬜🟩⬜\n🟩🟨⬜🟩🟨\n🟩🟩🟩🟩🟩
1521733010504445954    Back in the saddle.\n\nWordle 319 3/6\n\n⬜🟩⬜🟨⬜...
1521733001981444096    @RwellsWells Same. Are we both spotters? Haha ...
Name: text, dtype: object

Beautiful wordles! And the size is only about 22 MB. What a dream.

In [12]:
%%time
tweets_df.to_csv(merged_data_path + 'merged_tweets.csv')

Wall time: 4.58 s


Now do the same for the users.

## Merging and saving user data


In [180]:
users_df = get_merged_data(kind = 'users', numbers = range(18))

In [132]:
users_df.to_csv(merged_data_path + 'merged_users.csv')

In [11]:
test_users = get_raw_data('users', 82)
test_users

,Unnamed: 0,verified,description,protected,url,public_metrics,entities,profile_image_url,pinned_tweet_id,name,username,location
id,,,,,,,,,,,,
1.205522e+18,0,False,"丑年親子🐮#2021sep_baby/IVFを経て出産/実家疎遠,夫育休なしワンオペ/産後ク...",False,https://t.co/LYAG0uAH4e,"{'followers_count': 124, 'following_count': 12...","{'url': {'urls': [{'start': 0, 'end': 23, 'url...",https://pbs.twimg.com/profile_images/143827792...,1.472504e+18,ぽんた🥨7m,4sVB6b15O8xvbKR,NaN
1.508804e+08,1,False,NaN,False,NaN,"{'followers_count': 286, 'following_count': 68...",NaN,https://pbs.twimg.com/profile_images/113467789...,NaN,Héctor García,SoyElDelRincon,"Bogotá, Colombia"
1.218358e+18,2,False,🏳️‍🌈 || multifandom || artist || nonbinary dav...,False,NaN,"{'followers_count': 82, 'following_count': 365...","{'description': {'mentions': [{'start': 66, 'e...",https://pbs.twimg.com/profile_images/152126533...,1.514012e+18,## f: kurt || aftonbuilt aftonbuilt,lemondemonade,it/its he/him
2.872386e+09,3,False,ISTJ | In-game name: Somtuiz | Dormire quam su...,False,NaN,"{'followers_count': 9249, 'following_count': 2...","{'description': {'hashtags': [{'start': 55, 'e...",https://pbs.twimg.com/profile_images/150960011...,6.796901e+17,สมตุ้ย ,st_runth,NaN
2.527384e+09,4,False,創作とかゲームとかTRPGとか,False,NaN,"{'followers_count': 95, 'following_count': 102...",NaN,https://pbs.twimg.com/profile_images/137981432...,NaN,からすば,karasu947,お布団
...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,1288670750086279168,False,taylor swift loml,False,NaN,"{'followers_count': 211, 'following_count': 17...",NaN,https://pbs.twimg.com/profile_images/150035841...,1.455326e+18,gia (taylor’s version),vsmissdior,she her
NaN,1031551323831853056,False,steLLa☆後衛\r\nメモ欄:,False,https://t.co/8lydc7cleT,"{'followers_count': 67, 'following_count': 107...","{'url': {'urls': [{'start': 0, 'end': 23, 'url...",https://pbs.twimg.com/profile_images/146066217...,1.388144e+18,ﾉﾚイン≠レイン,RainCoat_Cloud,We're all under one sky.
NaN,20933313,False,"Knitting sorceress, mom, teacher, French horni...",False,https://t.co/1DLnaTjlrE,"{'followers_count': 1186, 'following_count': 1...","{'url': {'urls': [{'start': 0, 'end': 23, 'url...",https://pbs.twimg.com/profile_images/122594417...,1.172737e+18,Alison Coe 🏴‍☠️🧶,PirateKnits,"Long Island, NY"


### Fixing user data ids
the user data ids are messed up. I wrote about that a bit above.

These need to be resaved. The `Unnamed: 0` column is partly indices 0-99, and the index is id, while later they switch. In fact, the ids are all NaNs later on! 

It does seem like it's not reliably always exactly the first 100 with these indices, so I will make sure to check appropriately.

- for each user_xx.csv,
    - pull it in
    - check through all non-NaN indices. for each,
        - copy the index into the Unnamed: 0 column
        - reindex on Unnamed: 0
        - verify that all indices are unique, nothing is lost
        - save an updated copy

In [6]:
# fix user indexes.
# pulls in `users_{number}.csv, fixes the index so that it is the user id, and saves a new copy.

raw_data_path = '../raw_data/'
old_user_data_path = raw_data_path


def fix_user_index(number, extension = '_idfixed'):
    this_user_df = pd.read_csv(old_user_data_path + f'users_{number}.csv', index_col=None)
    this_user_df['id'] = this_user_df.apply(lambda row: row['Unnamed: 0'] if row.isna()['id'] else row['id'], axis = 1)
    this_user_df.set_index('id', inplace = True)
    this_user_df.drop(columns = 'Unnamed: 0', inplace = True)
    #save in the raw data path. do not overwrite the originals.
    this_user_df.to_csv(raw_data_path + f'users{extension}_{number}.csv')
    return None
    

In [62]:
%%time
# fix all of the user data.
first_users_number = 0
last_users_number = 84
for user_number in range(first_users_number, last_users_number + 1):
    fix_user_index(user_number)
print("done!")

done!
Wall time: 20.2 s


### Merging and saving user data

In [7]:
%%time
users_df = get_merged_data(kind = 'users_idfixed', numbers = range(84+1))

Wall time: 5.79 s


In [9]:
users_df

,name,public_metrics,url,description,profile_image_url,location,protected,username,verified,entities,pinned_tweet_id,withheld
id,,,,,,,,,,,,
1.006111e+18,Jose Antonio,"{'followers_count': 147, 'following_count': 43...",NaN,"Antiguo usuario bloqueado por error, pero he v...",https://pbs.twimg.com/profile_images/144681186...,"Alcalá de Guadaíra, Spain",False,Joseamj_97,False,NaN,NaN,NaN
3.600020e+08,Ana V Garcia,"{'followers_count': 41, 'following_count': 136...",https://t.co/uMfikCmd0I,NaN,https://pbs.twimg.com/profile_images/122215871...,Alcalá de Henares,False,AnaVGarciaG,False,"{'url': {'urls': [{'start': 0, 'end': 23, 'url...",NaN,NaN
8.288181e+17,Braden Van't Hof,"{'followers_count': 8, 'following_count': 282,...",NaN,LoL Head Coach/Manager/Mid Laner/4x Positional...,https://pbs.twimg.com/profile_images/149527273...,"Grand Rapids, MI",False,CLVRockmelon,False,NaN,NaN,NaN
8.620779e+08,John Milton ✊,"{'followers_count': 1465, 'following_count': 2...",https://t.co/iOfRLxEru5,"It's not criticism, it's just a different way ...",https://pbs.twimg.com/profile_images/150123920...,Spain,False,Geordioca,False,"{'url': {'urls': [{'start': 0, 'end': 23, 'url...",NaN,NaN
5.311651e+07,christopher jones,"{'followers_count': 189, 'following_count': 26...",NaN,I don't like cricket. oh no. i love it!,https://pbs.twimg.com/profile_images/110881711...,sheffield,False,chrisjones792,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2.988248e+09,a143753,"{'followers_count': 276, 'following_count': 16...",NaN,"2015年初ブルベ．'16,'17,'18,'19 SR．PBP2019 完走",https://pbs.twimg.com/profile_images/137075911...,Japan,False,a143753,False,NaN,NaN,NaN
2.992397e+09,にるよめ＠CGF🛡️💎🍃🦉,"{'followers_count': 244, 'following_count': 32...",https://t.co/Egk7FhvO32,2013/3/27に終了を迎えたMMORPG「コンチェルトゲート フォルテ」の世界で暮らして...,https://pbs.twimg.com/profile_images/150356943...,シルト＠ハウト密林地帯,False,nirutanmaiwaifu,False,"{'url': {'urls': [{'start': 0, 'end': 23, 'url...",1.035905e+18,NaN
3.357953e+07,Guillermo Crisóstomo,"{'followers_count': 986, 'following_count': 20...",NaN,De sangre altiva y noble corazón.,https://pbs.twimg.com/profile_images/151652867...,"Santiago - Quirihue, Chile",False,GCrisostomoT,False,NaN,1.449118e+18,NaN


In [89]:
len(tweets_df)

169574

170 thousand users and tweets in 85 files. A few more tweets than users. (Good! This means some users posted more than once, but not many)

In [90]:
merged_data_path

'../merged_data/'

In [11]:
users_df.to_csv(merged_data_path + 'merged_users.csv')

# Saving Wordle text

I want to grab the wordle grids out of the text, and convert them to arrays in their own dataframe. These should be indexed by the original tweet so they can be merged later.

Some of this was already done in a previous notebook Notebook1 in the cardle folder.

In [ ]:
# turn a row of squares into 